# CT dataset spliter

In [1]:
import pandas as pd, numpy as np, os

CT_TYPE = "NCCT"

if CT_TYPE == "CTA":
    N1_test, N0_test = 10,19
elif CT_TYPE == "NCCT":
    N1_test, N0_test = 14,33
else: assert False

data = pd.read_csv("../../data/gravo/table_data.csv")
pd.options.display.max_rows = 1000

## CT patients (with rankin)

In [2]:
ct_patients = data[(data[CT_TYPE] == "OK") & ~np.isnan(data["binary_rankin"])]

print("dataset size:", len(ct_patients))
print("rankin distr:", np.unique(ct_patients["binary_rankin"].values, return_counts = 1))

dataset size: 473
rankin distr: (array([0., 1.]), array([332, 141]))


## Create `dataset_{CT_TYPE}.csv`

In [3]:
ct_rankin1 = data[(data[CT_TYPE] == "OK") & (data["binary_rankin"] == 1)]["idProcessoLocal"].values
ct_rankin0 = data[(data[CT_TYPE] == "OK") & (data["binary_rankin"] == 0)]["idProcessoLocal"].values

np.random.seed()
np.random.shuffle(ct_rankin1)
np.random.shuffle(ct_rankin0)

with open(f"dataset_{CT_TYPE}.csv", "w") as f:
    f.write("patient_id,rankin,binary_rankin,set\n")
    for _, row in ct_patients.iterrows():
        f.write(f"{int(row['idProcessoLocal'])},{int(row['rankin'])},{int(row['binary_rankin'])},")
        if row["idProcessoLocal"] in ct_rankin1:
            f.write(f"{['train','test'][int(N1_test > 0)]}\n")
            N1_test -= 1
        else:
            f.write(f"{['train','test'][int(N0_test > 0)]}\n")
            N0_test -= 1

## Move `dataset_{CT_TYPE}.csv`

In [4]:
os.system(f"mv dataset_{CT_TYPE}.csv ../../data/gravo/")

0